In [78]:
import requests
from datetime import datetime, timedelta, UTC
import pandas as pd
from io import BytesIO
import json
from collections import defaultdict

session = requests.Session()
url = "http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/api/v1/"
url = "http://localhost:5002/api/v1/"

def api_get(path, *args, **kwargs):
    r = session.get(url + path, stream=True,*args, **kwargs)
    if not r.ok:
        print(f"API Error")
        print(json.dumps(r.json(), indent = 4))
    return r

In [79]:
api_get("list", params = {
}).json()

[{'mars_request': {'class': 'rd',
   'expver': 'xxxx',
   'stream': 'lwda',
   'aggregation_type': 'by_time',
   'date': '20250117',
   'platform': 'smart_citizen_kit',
   'time': '1300',
   'version': '1'},
  'url': '/api/v1/retrieve?class=rd&expver=xxxx&stream=lwda&aggregation_type=by_time&date=20250117&platform=smart_citizen_kit&time=1300&version=1',
  'datetime': '2025-01-17T13:00:00Z'},
 {'mars_request': {'class': 'rd',
   'expver': 'xxxx',
   'stream': 'lwda',
   'aggregation_type': 'by_time',
   'date': '20250117',
   'platform': 'acronet',
   'time': '1300',
   'version': '1'},
  'url': '/api/v1/retrieve?class=rd&expver=xxxx&stream=lwda&aggregation_type=by_time&date=20250117&platform=acronet&time=1300&version=1',
  'datetime': '2025-01-17T13:00:00Z'},
 {'mars_request': {'class': 'rd',
   'expver': 'xxxx',
   'stream': 'lwda',
   'aggregation_type': 'by_time',
   'date': '20250117',
   'platform': 'meteotracker',
   'time': '1700',
   'version': '1'},
  'url': '/api/v1/retrieve?

In [80]:
request = {'class': 'rd',
   'expver': 'xxxx',
   'stream': 'lwda',
   'aggregation_type': 'by_time',
   'date': '20250117',
   'platform': 'meteotracker',
   'time': '1700',
   'version': '1'}

data = api_get("retrieve", params = request | dict(format = "csv"))
pd.read_csv(data.raw)

,platform,author,station_id,external_station_id,station_name,aggregation_type,chunk_date,chunk_time,lat,lon,...,solar_radiation_index,air_pressure_near_surface,altitude,air_temperature_near_surface,dew_point_temperature,potential_temperature,humidity_index,vertical_temperature_gradient,living_lab,bluetooth_RSSI
0,meteotracker,Fredrik Wetterhall,8782b9a6b083c581,678a87c5a755720375a7f24e,meteotracker: 8782b9a6b083c581,by_time,20250117,1700,51.440075,-0.943189,...,NaN,1028.0,75.0,4.5,4.5,275.5,3.5,NaN,unknown,NaN
1,meteotracker,anonymous,df756fe03aece088,678a821da755720375a44be9,meteotracker: df756fe03aece088,by_time,20250117,1700,47.382140,9.662060,...,0.0,984.0,-635.0,1.6,-3.2,276.0,-1.1,NaN,unknown,-86.0
2,meteotracker,Andrea Ferrando,8b4614d626b8c4dd,678a866ca755720375a72cf7,meteotracker: 8b4614d626b8c4dd,by_time,20250117,1700,46.333442,10.060444,...,0.0,910.0,1034.0,-1.5,-7.3,279.1,-4.9,-1.848251,unknown,-73.0
3,meteotracker,anonymous,df756fe03aece088,678a821da755720375a44be9,meteotracker: df756fe03aece088,by_time,20250117,1700,47.382067,9.662165,...,0.0,984.0,-635.0,1.6,-3.2,276.0,-1.1,NaN,unknown,-86.0
4,meteotracker,Votre météo du doubs en direct,b89b4fdd474174ce,678a806ca755720375a35081,meteotracker: b89b4fdd474174ce,by_time,20250117,1700,47.358672,6.399935,...,0.3,997.0,267.0,1.2,-1.4,274.6,-1.4,NaN,unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624,meteotracker,anonymous,df756fe03aece088,678a821da755720375a44be9,meteotracker: df756fe03aece088,by_time,20250117,1700,47.671180,10.411910,...,0.0,924.0,-121.0,-3.1,-4.5,276.2,-6.2,-2.500000,unknown,-85.0
5625,meteotracker,anonymous,df756fe03aece088,678a821da755720375a44be9,meteotracker: df756fe03aece088,by_time,20250117,1700,47.670880,10.412130,...,0.0,924.0,-118.0,-3.1,-4.5,276.2,-6.2,-2.500000,unknown,-86.0
5626,meteotracker,marcofatelo,681316b84ab98399,678a9612a755720375aea499,meteotracker: 681316b84ab98399,by_time,20250117,1700,40.516015,-8.545053,...,0.0,1025.0,30.0,11.5,11.1,282.6,13.1,NaN,unknown,NaN
5627,meteotracker,anonymous,df756fe03aece088,678a821da755720375a44be9,meteotracker: df756fe03aece088,by_time,20250117,1700,47.670570,10.412350,...,0.0,924.0,-118.0,-3.1,-4.5,276.2,-6.2,-2.500000,unknown,-87.0


In [82]:
def sort_by_platform(stations) -> dict[str, list[dict]]:
    by_platform = defaultdict(list)
    for s in stations:
        by_platform[s["platform"]].append(s)
    return by_platform
    
lookback = timedelta(hours = 5) 

all_stations = api_get("stations", params = {
}).json()

recent_stations = api_get("stations", params = {
    "start_time" : datetime.now(UTC) - lookback
}).json()

print("All stations")
all_by_platform = sort_by_platform(all_stations)
print({k : len(v) for k, v in all_by_platform.items()})
print()

print(f"Recent ({lookback})")
print({k : len(v) for k, v in sort_by_platform(recent_stations).items()})
print()

API Error


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Total Meteotracker datasets

In [ ]:
all_by_platform['meteotracker'][-1]

In [ ]:
dates = [datetime.fromisoformat(s['time_span']["end"]) for s in all_by_platform['meteotracker']]
earliest, latest = min(dates), max(dates)
print(f"Meteotracker: {len(all_by_platform['meteotracker'])} tracks.")
print(f"Earliest ingested data: {earliest}")
print(f"Most recent ingested data: {latest}")

## Total Acronet datasets

In [ ]:
def get_all_granules_by_platform(platform):
    granules = api_get("list", params = {'platform': platform}).json()
    dates = [datetime.fromisoformat(d["datetime"]) for d in granules]
    print(set(g["mars_request"]["platform"] for g in granules))
    # assert all(g["mars_request"]["platform"] == platform for g in granules)
    return granules, dates

acronet_granules, dates = get_all_granules_by_platform(platform = "acronet")
earliest, latest = min(dates), max(dates)
total_time = timedelta(minutes = 5) * len(acronet_granules)

print(f"Acronet: {len(all_by_platform['acronet'])} distinct stations.")
print(f"Earliest ingested data: {earliest} Most recent ingested data: {latest}")
print(f"Ingested data: {len(acronet_granules)} 5 minute acronet data granules.")
print(f"Total time: {total_time}")

## Total Smart Citizen Kit Datasets

In [ ]:
sck_granules, dates = get_all_granules_by_platform(platform = "smart_citizen_kit")
earliest, latest = min(dates), max(dates)
total_time = timedelta(minutes = 5) * len(sck_granules)

print(f"Smart Citizen Kit: {len(all_by_platform['smart_citizen_kit'])} distinct stations.")
print(f"Earliest ingested data: {earliest} Most recent ingested data: {latest}")
print(f"Ingested data: {len(sck_granules)} 5 minute sck data granules.")
print(f"Total time: {total_time}")

## Retrieving chunked data for continuous data streams
Acronet, Smart Citizen Kit

In [ ]:
def plot_example_station(station):
    latest = datetime.fromisoformat(example_station["time_span"]["end"])
    print(f"Latest data: {latest}")
    args = {
        "start_time" : (latest - timedelta(hours = 12)).isoformat(),
        "end_time": latest.isoformat(),
    }
    
    data = api_get("retrieve", params = example_station["mars_request"] | args | dict(format = "csv"))
    df = pd.read_csv(BytesIO(data.content))
    
    df["datetime"] = pd.to_datetime(df["datetime"])
    exclude_columns = {"external_id", "date", "lat", "lon", "altitude"}
    numeric_columns = [col for col, dtype in zip(df.columns, df.dtypes) 
                       if (dtype == "float64" or dtype == "int64")
                       and col not in exclude_columns]
    
    axes = df.plot(
        x = "datetime",
        y = numeric_columns,
        subplots=True, layout=(len(numeric_columns), 1), figsize=(8, 2*len(numeric_columns)), sharex=True, rot=90)

example_station = all_by_platform["acronet"][-1]
plot_example_station(example_station)

## Retrieve and plot a Meteotracker track

In [ ]:
example_station = all_by_platform["meteotracker"][-1]

print(json.dumps(example_station["mars_request"], indent = 4))
granule_list = api_get("list", params = example_station["mars_request"]).json()

print(f"The above MARS request matches {len(granule_list)} data granule(s).")

# Sanity check
assert len(granule_list) == 1

args = {
    "format" : "json"
}

In [ ]:
data = api_get("retrieve", params = example_station["mars_request"] | args)
df = pd.DataFrame.from_records(data.json())
df["datetime"] = pd.to_datetime(df["datetime"])

exclude_columns = {"external_id", "date", "lat", "lon", "altitude"}
numeric_columns = [col for col, dtype in zip(df.columns, df.dtypes) 
                   if (dtype == "float64" or dtype == "int64")
                   and col not in exclude_columns]

axes = df.plot(
    x = "datetime",
    y = numeric_columns,
    subplots=True, layout=(len(numeric_columns), 1), figsize=(8, 2*len(numeric_columns)), sharex=True, rot=90)


In [ ]:
import geopandas as gpd
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
geo_df.explore(column = "altitude")